# SALES INSIGHT PROJECT

### Import Libraries 

In [1]:
import pandas as pd
import os
from matplotlib import pyplot as plt

### Import the database from the system
#### A zip file containing sales records for an entire year, divided into separate .csv files, was provided by the esteemed owner. Our initial objective was to consolidate all the data from the individual files into a single .csv file. As a result, we created a consolidated file named "all__data.csv" for further analysis and reference.

In [2]:
# df = pd.read_csv("E:/DATA SCIENCE/Data for projects/1Project Sales Analysis/Sales_Data/Sales_April_2019.csv")
# df.head()

files = [file for file in os.listdir('E:/DATA SCIENCE/Data for projects/1Project Sales Analysis/Sales_Data')]
# for f in files:
#     print(f)
# print(files)

all_month_data = pd.DataFrame()

for f in files:
     df = pd.read_csv("E:/DATA SCIENCE/Data for projects/1Project Sales Analysis/Sales_Data/" + f)
     all_month_data = pd.concat([all_month_data,df])
all_month_data.to_csv("all__data.csv", index=False)

In [3]:
all_data = pd.read_csv("all__data.csv")
all_data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,NaN,NaN,NaN,NaN,NaN,NaN
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"


### Profile Report

#### Through this Profile Report, I aim to conduct an initial analysis to gain a better understanding of the dataset. This report will provide valuable insights into the dataset's structure, missing values, data types, and other relevant information. By exploring these aspects, I can lay the foundation for further analysis and decision-making based on a comprehensive understanding of the dataset.

In [4]:
from pandas_profiling import ProfileReport
prof = ProfileReport(all_data)
prof.to_file(output_file = "output.html")

C:\Users\DELL\anaconda3\lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
C:\Users\DELL\anaconda3\lib\site-packages\visions\backends\shared\nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:
C:\Users\DELL\AppData\Local\Temp\ipykernel_24996\3750828349.py:1: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st.

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Cleanup Data!

#### Before delving into the analysis phase, we encountered certain anomalies in the data, such as the presence of 'NaN' values and 'Or' values in the Order Date column. Although we addressed these issues at the beginning of our analysis, it is possible that we may need to handle them again during the analysis process.

##### Removing all NaN values from database system

In [5]:
nan_rows = all_data[all_data.isna().any(axis=1)]
nan_rows.head()


,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
1,NaN,NaN,NaN,NaN,NaN,NaN
356,NaN,NaN,NaN,NaN,NaN,NaN
735,NaN,NaN,NaN,NaN,NaN,NaN
1433,NaN,NaN,NaN,NaN,NaN,NaN
1553,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
all_data = all_data.dropna()

##### Find 'Or' and delete it 

In [7]:
all_data = all_data[all_data['Order Date'].str[:2] != 'Or']

##### Adding a Month Column

In [8]:
all_data['Month'] = all_data['Order Date'].str[:2]
all_data['Month'] = all_data['Month'].apply(pd.to_numeric)
all_data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001",4
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215",4
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001",4
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001",4
5,176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001",4


##### Convert column to the correct type

In [9]:
all_data['Quantity Ordered'] = all_data['Quantity Ordered'].apply(pd.to_numeric)
all_data['Price Each'] = all_data['Price Each'].apply(pd.to_numeric)

print(all_data.dtypes)

Order ID             object
Product              object
Quantity Ordered      int64
Price Each          float64
Order Date           object
Purchase Address     object
Month                 int64
dtype: object


#### Question 1: What was the best month for sale? How much was earned that month?

#### In order to address this question, our first step was to determine the total sales for each specific month. To accomplish this, we augmented the dataframe by adding a new column titled 'Total Sales', which would facilitate our objective.

##### Add the Sales column

In [10]:
all_data['Total Sales'] = all_data['Quantity Ordered'] * all_data['Price Each']
all_data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Total Sales
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001",4,23.90
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215",4,99.99
3,176560,Google Phone,1,600.00,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001",4,600.00
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001",4,11.99
5,176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001",4,11.99


In [11]:
sum1 = all_data.groupby('Month').sum()
sum1

C:\Users\DELL\AppData\Local\Temp\ipykernel_24996\511680508.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sum1 = all_data.groupby('Month').sum()


,Quantity Ordered,Price Each,Total Sales
Month,,,
1,10903,1811768.38,1822256.73
2,13449,2188884.72,2202022.42
3,17005,2791207.83,2807100.38
4,20558,3367671.02,3390670.24
5,18667,3135125.13,3152606.75
6,15253,2562025.61,2577802.26
7,16072,2632539.56,2647775.76
8,13448,2230345.42,2244467.88
9,13109,2084992.09,2097560.13


In [12]:
sum1.max()

Quantity Ordered      28114.00
Price Each          4588415.41
Total Sales         4613443.34
dtype: float64

#### Answer 1: Best Month for sales was December and sales for that month is 4613443.34 US Dollars

In [13]:
Months = range(1,13)
plt.title('Sales Analysis')
plt.xlabel('Month')
plt.ylabel('Sales in USD($)')
plt.bar(Months,sum1['Total Sales'])
plt.grid()
plt.show()

C:\Users\DELL\AppData\Local\Temp\ipykernel_24996\1553193809.py:7: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


#### Question 2: Which city has the highest number of Sales? 

#### In order to extract the city information, we performed manipulation on the 'Purchase Address' column. However, during the analysis, we encountered an instance where different countries had a city with the same name. To ensure individuality for each unique element in the 'City' column, we incorporated the country code alongside the city name. This approach allowed us to differentiate cities located in different countries and maintain data integrity throughout the analysis.

##### Adding city column

In [14]:
def get_city(address):
    return address.split(',')[1]
def get_state(address):
    return address.split(',')[2].split(' ')[1]

all_data['City'] = all_data['Purchase Address'].apply(lambda x:get_city(x) + '('+ get_state(x) + ')')
all_data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Total Sales,City
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001",4,23.90,Dallas(TX)
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215",4,99.99,Boston(MA)
3,176560,Google Phone,1,600.00,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001",4,600.00,Los Angeles(CA)
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001",4,11.99,Los Angeles(CA)
5,176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001",4,11.99,Los Angeles(CA)


In [15]:
sum2 = all_data.groupby('City').sum()
sum2

C:\Users\DELL\AppData\Local\Temp\ipykernel_24996\2570848278.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sum2 = all_data.groupby('City').sum()


,Quantity Ordered,Price Each,Month,Total Sales
City,,,,
Atlanta(GA),16602,2779908.20,104794,2795498.58
Austin(TX),11153,1809873.61,69829,1819581.75
Boston(MA),22528,3637409.77,141112,3661642.01
Dallas(TX),16730,2752627.82,104620,2767975.40
Los Angeles(CA),33289,5421435.23,208325,5452570.80
New York City(NY),27932,4635370.83,175741,4664317.43
Portland(ME),2750,447189.25,17144,449758.27
Portland(OR),11303,1860558.22,70621,1870732.34
San Francisco(CA),50239,8211461.74,315520,8262203.91


In [16]:
sum2.max()

Quantity Ordered      50239.00
Price Each          8211461.74
Month                315520.00
Total Sales         8262203.91
dtype: float64

#### Answer 2: Maximum Sale is in the San Francisco with Total Sales 8262203.91 USD

In [17]:
cities = [city for city, df in all_data.groupby('City')]

plt.bar(cities, sum2['Total Sales'])
plt.xticks(rotation=90)
plt.xlabel('Cities')
plt.ylabel('Sales in USD')

Text(0, 0.5, 'Sales in USD')

#### Question 3: What time should we display advertisements to maximize likelihood of customer's buying product?

#### To determine the optimal time for maximizing their advertising efforts, we needed to extract the exact hours from the 'Order Date' column. This information would enable us to identify the approximate hours during which they should focus their advertising activities.

In [18]:
all_data['Order Date'] = pd.to_datetime(all_data['Order Date'])

In [19]:
all_data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Total Sales,City
0,176558,USB-C Charging Cable,2,11.95,2019-04-19 08:46:00,"917 1st St, Dallas, TX 75001",4,23.90,Dallas(TX)
2,176559,Bose SoundSport Headphones,1,99.99,2019-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215",4,99.99,Boston(MA)
3,176560,Google Phone,1,600.00,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",4,600.00,Los Angeles(CA)
4,176560,Wired Headphones,1,11.99,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",4,11.99,Los Angeles(CA)
5,176561,Wired Headphones,1,11.99,2019-04-30 09:27:00,"333 8th St, Los Angeles, CA 90001",4,11.99,Los Angeles(CA)


In [20]:
all_data['Hour'] = pd.to_datetime(all_data['Order Date']).dt.hour
all_data['Minute'] = pd.to_datetime(all_data['Order Date']).dt.minute
# all_data = all_data.drop("Time", axis=1)
# all_data = all_data.drop("Date", axis=1)

In [21]:
all_data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Total Sales,City,Hour,Minute
0,176558,USB-C Charging Cable,2,11.95,2019-04-19 08:46:00,"917 1st St, Dallas, TX 75001",4,23.90,Dallas(TX),8,46
2,176559,Bose SoundSport Headphones,1,99.99,2019-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215",4,99.99,Boston(MA),22,30
3,176560,Google Phone,1,600.00,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",4,600.00,Los Angeles(CA),14,38
4,176560,Wired Headphones,1,11.99,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",4,11.99,Los Angeles(CA),14,38
5,176561,Wired Headphones,1,11.99,2019-04-30 09:27:00,"333 8th St, Los Angeles, CA 90001",4,11.99,Los Angeles(CA),9,27


#### Answer 3: Based on the graph below, we can draw the conclusion that around 12 noon and 7:00 PM are the optimal times for the company to conduct marketing activities. By strategically targeting these specific hours, they can effectively enhance their sales while minimizing advertising costs throughout the entire day.

In [22]:
hours = [hour for hour, df in all_data.groupby('Hour')]
plt.plot(hours,all_data.groupby(['Hour']).count())
all_data.groupby(['Hour']).count()
plt.xticks(hours)
plt.xlabel('Hours')
plt.ylabel('Number of Orders')
plt.grid()
plt.show()

C:\Users\DELL\AppData\Local\Temp\ipykernel_24996\2882060331.py:8: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


#### Question 4: What products that are most often sold together?

#### To get the products that are sold together we first combine the rows having same 'Order ID' and 'Purchase Address'.

In [23]:
all_data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Total Sales,City,Hour,Minute
0,176558,USB-C Charging Cable,2,11.95,2019-04-19 08:46:00,"917 1st St, Dallas, TX 75001",4,23.90,Dallas(TX),8,46
2,176559,Bose SoundSport Headphones,1,99.99,2019-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215",4,99.99,Boston(MA),22,30
3,176560,Google Phone,1,600.00,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",4,600.00,Los Angeles(CA),14,38
4,176560,Wired Headphones,1,11.99,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",4,11.99,Los Angeles(CA),14,38
5,176561,Wired Headphones,1,11.99,2019-04-30 09:27:00,"333 8th St, Los Angeles, CA 90001",4,11.99,Los Angeles(CA),9,27


In [24]:
df = all_data[all_data.duplicated(['Order ID'], keep=False)]
df['Grouped'] = df.groupby('Order ID')['Product'].transform(lambda x:','.join(x))
df = df[['Order ID', 'Grouped']].drop_duplicates()
df.head()

C:\Users\DELL\AppData\Local\Temp\ipykernel_24996\3491274382.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Grouped'] = df.groupby('Order ID')['Product'].transform(lambda x:','.join(x))


,Order ID,Grouped
3,176560,"Google Phone,Wired Headphones"
18,176574,"Google Phone,USB-C Charging Cable"
30,176585,"Bose SoundSport Headphones,Bose SoundSport Hea..."
32,176586,"AAA Batteries (4-pack),Google Phone"
119,176672,"Lightning Charging Cable,USB-C Charging Cable"


In [25]:
 # Reference - https://stackoverflow.com/questions/52195887/counting-unique-pairs-of-numbers-into-a-python-dictionary
from itertools import combinations
from collections import Counter

count = Counter()

for row in df['Grouped']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list,2)))
count.most_common(10)

[(('iPhone', 'Lightning Charging Cable'), 1005),
 (('Google Phone', 'USB-C Charging Cable'), 987),
 (('iPhone', 'Wired Headphones'), 447),
 (('Google Phone', 'Wired Headphones'), 414),
 (('Vareebadd Phone', 'USB-C Charging Cable'), 361),
 (('iPhone', 'Apple Airpods Headphones'), 360),
 (('Google Phone', 'Bose SoundSport Headphones'), 220),
 (('USB-C Charging Cable', 'Wired Headphones'), 160),
 (('Vareebadd Phone', 'Wired Headphones'), 143),
 (('Lightning Charging Cable', 'Wired Headphones'), 92)]

#### Answer 4: According to the analysis conducted, the products most frequently bought together are 'IPhone' and 'Lightning Charging Cable', while the second most commonly purchased combination consists of 'Google Phone' and 'USB-C Charging Cable'. These findings highlight the tendency of customers to purchase these products together, potentially indicating a complementary relationship between them.

#### Question 5: What product sold the most? Why do you think it sold the most?

In [26]:
all_data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Total Sales,City,Hour,Minute
0,176558,USB-C Charging Cable,2,11.95,2019-04-19 08:46:00,"917 1st St, Dallas, TX 75001",4,23.90,Dallas(TX),8,46
2,176559,Bose SoundSport Headphones,1,99.99,2019-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215",4,99.99,Boston(MA),22,30
3,176560,Google Phone,1,600.00,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",4,600.00,Los Angeles(CA),14,38
4,176560,Wired Headphones,1,11.99,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",4,11.99,Los Angeles(CA),14,38
5,176561,Wired Headphones,1,11.99,2019-04-30 09:27:00,"333 8th St, Los Angeles, CA 90001",4,11.99,Los Angeles(CA),9,27


In [27]:
sum3 = all_data.groupby('Product').sum()
sum3

C:\Users\DELL\AppData\Local\Temp\ipykernel_24996\1968223815.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sum3 = all_data.groupby('Product').sum()


,Quantity Ordered,Price Each,Month,Total Sales,Hour,Minute
Product,,,,,,
20in Monitor,4129,451068.99,29336,454148.71,58764,122252
27in 4K Gaming Monitor,6244,2429637.70,44440,2435097.56,90916,184331
27in FHD Monitor,7550,1125974.93,52558,1132424.50,107540,219948
34in Ultrawide Monitor,6199,2348718.19,43304,2355558.01,89076,183480
AA Batteries (4-pack),27635,79015.68,145558,106118.40,298342,609039
AAA Batteries (4-pack),31017,61716.59,146370,92740.83,297332,612113
Apple Airpods Headphones,15661,2332350.00,109477,2349150.00,223304,455570
Bose SoundSport Headphones,13457,1332366.75,94113,1345565.43,192445,392603
Flatscreen TV,4819,1440000.00,34224,1445700.00,68815,142789


In [28]:
Proc = [proc for proc, df in all_data.groupby('Product')]

plt.bar(Proc, sum3['Quantity Ordered'],color='green')
plt.xlabel('Products')
plt.ylabel('Quantity Sold')
plt.xticks(rotation = 90)

plt.show()

C:\Users\DELL\AppData\Local\Temp\ipykernel_24996\803710708.py:8: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


#### Prices of all the objects

In [29]:
sum3['Price (USD)'] = sum3['Price Each']/sum3['Quantity Ordered']
sum3 

,Quantity Ordered,Price Each,Month,Total Sales,Hour,Minute,Price (USD)
Product,,,,,,,
20in Monitor,4129,451068.99,29336,454148.71,58764,122252,109.244124
27in 4K Gaming Monitor,6244,2429637.70,44440,2435097.56,90916,184331,389.115583
27in FHD Monitor,7550,1125974.93,52558,1132424.50,107540,219948,149.135752
34in Ultrawide Monitor,6199,2348718.19,43304,2355558.01,89076,183480,378.886625
AA Batteries (4-pack),27635,79015.68,145558,106118.40,298342,609039,2.859261
AAA Batteries (4-pack),31017,61716.59,146370,92740.83,297332,612113,1.989767
Apple Airpods Headphones,15661,2332350.00,109477,2349150.00,223304,455570,148.927272
Bose SoundSport Headphones,13457,1332366.75,94113,1345565.43,192445,392603,99.009196
Flatscreen TV,4819,1440000.00,34224,1445700.00,68815,142789,298.817182


#### Answer 5: The 'AAA Batteries (4-pack)' stands out as the top-selling product based on the analysis conducted. Notably, I observed a trend where products with lower costs tend to exhibit higher sales, as evidenced by the graph below. Additionally, it is important to consider the nature of the products being sold. Monitors, laptops, Google Phones, and iPhones are not typically purchased on a daily basis. On the other hand, AAA Batteries are items that are in constant demand for everyday use. This factor may contribute to their consistently high sales figures compared to other products in the dataset.

In [30]:
prices = all_data.groupby('Product').mean()['Price Each']
# print(prices)

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.bar(Proc, sum3['Quantity Ordered'],color='green')
ax2.plot(Proc, prices, color='blue')

ax1.set_xlabel('Products')
ax1.set_ylabel('Quantity Sold', color='g')
ax2.set_ylabel('Price($)', color='b')
ax1.set_xticklabels(Proc,rotation='vertical')
plt.show()

C:\Users\DELL\AppData\Local\Temp\ipykernel_24996\2097078878.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  prices = all_data.groupby('Product').mean()['Price Each']
C:\Users\DELL\AppData\Local\Temp\ipykernel_24996\2097078878.py:12: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax1.set_xticklabels(Proc,rotation='vertical')
C:\Users\DELL\AppData\Local\Temp\ipykernel_24996\2097078878.py:13: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


### Conclusion

#### The provided sales data pertains to an Electronics Company and consists of records for 20 distinct products. Analyzing this dataset offers valuable insights into sales performance, customer preferences, and business dynamics within the electronics industry.

#### Our analysis reveals that customers show a higher inclination to make purchases during the last month of the year, possibly due to a year-end sale strategy aimed at selling older inventory at discounted prices to make room for new products in the upcoming year. New product launches in the early months may initially experience lower sales due to higher quoted prices.

#### San Francisco emerges as the location with the highest number of sales, indicating a receptive market for the electronics shop. Focusing marketing efforts on this market can further capitalize on its strong sales potential.

#### Peak sales occur around 12 noon and 7 PM, providing strategic opportunities for targeted advertising. By concentrating marketing activities during these hours, the shop owner can maximize sales while minimizing advertising costs throughout the day.

#### The most frequently bought product combinations include 'IPhone' and 'Lightning Charging Cable', followed by 'Google Phone' and 'USB-C Charging Cable', suggesting a complementary relationship between these products.

#### The top-selling product is the 'AAA Batteries (4-pack)', with lower-priced products exhibiting higher sales. Products like Monitors, laptops, Google Phones, and iPhones, which are not daily purchases, show relatively lower sales compared to the constant demand for AAA Batteries.

#### These insights provide valuable guidance for optimizing business strategies and decision-making.